# 강화학습을 사용하여 나만의 똑똑한(????) 슈퍼마리오 만들기


이제부터 DQN을 사용하여 장애물을 피해 Goal 까지 가는 마리오를 만들어 보도록 하겠습니다. 

이 코드는 example 코드로 봐주시고, 직접 "파이썬과 케라스로 배우는 강화학습"을 응용하시거나 직접 만드신 코드로 

똑똑한 슈퍼마리오를 만들어주세요 ^^

코드를 이제 막 시작한 컴맹인 저의 코드는 함정이 아주 많으니 틀린부분이나 개선해야할 부분 있으면 꼭 꼭 말씀 부탁드려요 ^^

DQN이외에도 본인이 원하시는 학습 알고리즘을 사용하셔도 무방합니다!


다시한번,

## 이 코드는 슈퍼마리오가 어떻게 환경과 상호작용하는지 참고만 하시고 더 좋은 코드를 만들어주세요 ^^ 

In [1]:

import gym
import ppaquette_gym_super_mario
import random
import numpy as np 
from wrapper import action_space
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Flatten, Input, Lambda, merge
from keras.optimizers import RMSprop
from keras.models import Model, Sequential
from skimage.color import rgb2gray
from collections import deque
from keras import backend as K
import tensorflow as tf
from sum_tree import SumTree



load_model = False



#DQN의 하이퍼파라미터 입니다. 


#epsilon 값은 처음에 1로 시작하여 학습이 진행될수록 0.1까지 계쏙 줄어듭니다. 
epsilon = 1.
epsilon_start, epsilon_end = 1, 0.1
exploration_steps = 400000.

epsilon_decay_step = (epsilon_start - epsilon_end) \
                                  / exploration_steps
batch_size = 32

discount_factor = 0.99

train_start= 10000
update_target_rate = 5000
#리플레이 메모리에 최대 100000 까지 저장합니다.  
memory=deque(maxlen=100000)
just_start_train=True
state_size = (84,84,4)
action_size =6
env = gym.make('ppaquette/meta-SuperMarioBros-v0')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Deep learning model

In [2]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32,(8,8),strides =(4,4), activation='relu', input_shape=state_size))
    model.add(Conv2D(64,(4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64,(3,3), strides =(1,1), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(action_size))
    model.summary()

    return model

# Define model and target model

In [3]:
model = build_model()
target_model=build_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total params: 1,687,206
Trainable params: 1,687,206
Non-trainable params: 0
_________________________________________________________________


### append <s,a,r,s'> at replay memory

# Functions 

### change action(number) to the action( action what I think I need )

### append <s,a,r,s'> at replay memory

In [4]:
import random




In [5]:
#Epsilon greedy action을 선택합니다.

action_size=6
def get_action(history):
        history = np.float32(history / 255.0)
        if np.random.rand() <= epsilon:
            return random.randrange(action_size)
        else:
            q_value = model.predict(history)
            return np.argmax(q_value[0])

In [6]:
#state, action, reward, next state의 정보를 memory에 append합니다

def append_sample(history, action, reward, next_history, dead):
    memory.append((history,action,reward,next_history,dead))

### Optimizer

In [7]:
def optimizer(self):
        a = K.placeholder(shape=(None,), dtype='int32')
        y = K.placeholder(shape=(None,), dtype='float32')

        prediction = model.output

        a_one_hot = K.one_hot(a, action_size)
        q_value = K.sum(prediction * a_one_hot, axis=1)
        error = K.abs(y - q_value)

        quadratic_part = K.clip(error, 0.0, 1.0)
        linear_part = error - quadratic_part
        loss = K.mean(0.5 * K.square(quadratic_part) + linear_part)

        optimizer = RMSprop(lr=0.00025, epsilon=0.01)
        updates = optimizer.get_updates(model.trainable_weights, [], loss)
        train = K.function([model.input, a, y], [loss], updates=updates)

        return train

### Update models 


In [8]:
def update_target_model():
        target_model.set_weights(model.get_weights())

## Train model
(Batch from the memory)

In [9]:
epsilon=0.9

def train_model():
    global epsilon
    if epsilon > epsilon_end:
        epsilon -= epsilon_decay_step
    
    mini_batch= random.sample(memory, batch_size)
   
    
  
    history = np.zeros((batch_size, state_size[0],
                            state_size[1], state_size[2]))
    next_history = np.zeros((batch_size, state_size[0],
                             state_size[1], state_size[2]))
    target = np.zeros((batch_size,))
    action, reward, dead = [], [], []

    for i in range(batch_size):
            history[i] = np.float32(mini_batch[i][0] / 255.)
            next_history[i] = np.float32(mini_batch[i][3] / 255.)
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            dead.append(mini_batch[i][4])

    target_value = target_model.predict(next_history)

    for i in range(batch_size):
        if dead[i]:
            target[i] = reward[i]
        else:
            target[i] = reward[i] + discount_factor * \
                                        np.amax(target_value[i])

    loss = optimizer([history, action, target])

    
    

# 그럼 학습을 시작합니다. 

In [10]:

    

###### 환경을 불러옵니다.

wra_act=action_space
scores, episodes, global_step = [], [], 0

#reduce actions
env=wra_act.mario_action(env)

#reduce inputsize to 84,84,1
env=wra_act.ProcessFrame84(env)

step=0
obs=env.reset()


for e in range(100000):
        done = False
        
       
        
        
        
        reshape_obs=np.reshape([obs],(1,84,84,1))
        history=np.stack((obs,obs,obs,obs), axis = 2)
        
        history = np.reshape([history], (1, 84, 84, 4))


        while not done:
           
            global_step += 1
            step += 1

            """
                0: [0, 0, 0, 0, 0, 0],  # NOOP
                1: [1, 0, 0, 0, 0, 0],  # Up
                2: [0, 0, 1, 0, 0, 0],  # Down
                3: [0, 1, 0, 0, 0, 0],  # Left
                4: [0, 1, 0, 0, 1, 0],  # Left + A
                5: [0, 1, 0, 0, 0, 1],  # Left + B
                6: [0, 1, 0, 0, 1, 1],  # Left + A + B
                7: [0, 0, 0, 1, 0, 0],  # Right
                8: [0, 0, 0, 1, 1, 0],  # Right + A
                9: [0, 0, 0, 1, 0, 1],  # Right + B
                10: [0, 0, 0, 1, 1, 1],  # Right + A + B
                11: [0, 0, 0, 0, 1, 0],  # A
                12: [0, 0, 0, 0, 0, 1],  # B
                13: [0, 0, 0, 0, 1, 1],  # A + B
                6개의 버튼으로 14개의 action을 조합할수 있습니다. 
                좀 더 빠른 학습을 위해 필요하지 않은 action은 없애주셔도 좋습니다. ( 예를 들어 Up action, 아무것도 하지 않는 action) 
                
                아래의 action 선택은 본인이 생각하셨을때 필요한 action들로 바꿔보세요 ^^ action size를 더 줄이셔도, 늘리셔도 무관합니다. 
            """
            #바로 전 4개의 state로 action을 선택합니다.
            
            action=get_action(history)
            if action==0:
                actions=[0,0,0,0,0,0]
            elif action==1:
                actions=[1,0,0,0,0,0]
    
            elif action==2:
                actions=[0,1,0,0,0,0]
            elif action==3:
                actions=[0,0,1,0,0,0]
            elif action==4:
                actions=[0,0,0,1,0,0]
    
            elif action==5:
                actions=[0,0,0,0,1,0]
            elif action==6:
                actions=[0,0,0,0,0,1]
    
            
            # 선택한 행동으로 환경에서 한 타임스텝 진행
            observe, reward, done, clear= env.step(actions)
            
            if clear:
                reward += 30
                done = True
            #끝났지만 clear 하지 못했다면 
            if done and not clear:
                reward = -30
              
            reward = np.clip(reward, -1., 1.)
               
    
            next_state = observe
            
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history = np.append(next_state, history[:, :, :, :3], axis=3)
            
            

            
            #sample <s,a,r,s'>를 리플레이 메모리에 저장 후 학습
            
            append_sample(history, action, reward, next_history, done)

    

            if len(memory)>= train_start:
                train_model()
                
            if global_step % update_target_rate ==0:
                update_target_model()
            
            history = next_history
            
            #모델의 weight를 저장합니다. 
 
            if e % 100 == 0:
                model.save_weights("./save/manual.h5")
            

KeyboardInterrupt: 